In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'head_diameter_sae'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['image_url']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

2614


In [5]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [6]:
pat_mm=rf'''((?<=\d)\s?\-?mm)|()'''         
df['mm_name'] = df['product_name'].apply(lambda x: re_extract(pat_mm, str(x)))
df['mm_ld'] = df['long_desc'].apply(lambda x: re_extract(pat_mm, str(x)))
df['mm_hd'] = df['HeadDiameter'].apply(lambda x: re_extract(pat_mm, str(x)))

pat_hd=r'(?i)(?i)(Head Diameter)|()'
df['head']=df['long_desc'].apply(lambda x: re_extract(pat_hd, x))

In [7]:
df[df['HeadDiameter'].astype(str)!='nan']['HeadDiameter'].explode().value_counts()

20mm      75
13mm      57
18mm      45
15mm      34
16mm      32
14mm      20
12mm      15
10mm      14
17mm      13
30mm      10
25mm       6
22mm       6
19mm       6
9mm        6
11mm       5
12.5mm     4
21mm       3
8mm        2
6.5mm      2
9.5mm      2
13.5mm     1
17.5mm     1
24mm       1
26mm       1
11.5mm     1
23mm       1
6.3mm      1
32mm       1
28mm       1
8.5mm      1
38mm       1
16.5mm     1
7.5mm      1
36.5mm     1
25/64"     1
Name: HeadDiameter, dtype: int64

# N/A

In [8]:
na=df[(df['HeadDiameter'].astype(str)=='nan')&(df['head'].astype(str)=='[]')]
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
print(len(na))

2164


# Curated Fields

In [9]:
print(len(df[(df['HeadDiameter'].astype(str)!='nan')&(df['mm_hd'].astype(str)=='[]')]))
curate=df[(df['HeadDiameter'].astype(str)!='nan')&(df['mm_hd'].astype(str)=='[]')]
curate['hd']=curate['HeadDiameter'].apply(lambda x: f'["{x}"]').apply(lambda x: re.sub(r'(?<=\d)"',' in',str(x)))
curate[curation_col]=curate['hd']
matchcurate=curate[['external_id',curation_col]]
from natsort import natsorted
matchcurate[curation_col]=matchcurate[curation_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
rounds(matchcurate,curation_col)
matchcurate[curation_col].explode().value_counts()

1


["25/64 in"]    1
Name: Q:head_diameter_sae, dtype: int64

# Name/Description

In [10]:
pat_hd=r'''(?i)(Head Diameter\:?\s?(?:\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?|\d+\/\d+)(?:"|”))|()'''
df['hd_ld']=df['long_desc'].apply(lambda x: re_extract(pat_hd, x))

ex1=df[(df['head'].astype(str)!='[]')&(df['hd_ld'].astype(str)!='[]')]
ex=ex1
ex

ex['hd_ld']=ex['hd_ld'].apply(lambda x: re.sub(r'(?i)(Head Diameter\:?\s?)','',str(x))).apply(lambda x: re.sub(r'(?:"|”)',' in',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))#.apply(lambda x: re.sub(r"(?i)'",'"',str(x)))
ex[curation_col]=ex['hd_ld']
matchex=ex[['external_id',curation_col]]
from natsort import natsorted
matchex[curation_col]=matchex[curation_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
rounds(matchex,curation_col)
matchex[curation_col].explode().value_counts()

["29/64 in"]    4
["3/4 in"]      2
["9/16 in"]     2
["21/32 in"]    1
["19/32 in"]    1
["13/32 in"]    1
["11/16 in"]    1
["1/2 in"]      1
Name: Q:head_diameter_sae, dtype: int64

# Final N/A

In [12]:
final=ex1[(ex1['HeadDiameter'].astype(str)=='nan')&(ex1['hd_ld'].astype(str)=='[]')]
# print(len(final))

print(len(final[final['head'].astype(str)=='[]']))
final=final[final['head'].astype(str)=='[]']
final[curation_col]='n/a'
matchfinal=final[['external_id',curation_col]]
final

0


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, keywords, packageweight, packagequantity, shortdescription, Type, _ofPieces, PrivateLabel, Finish, HeadStyle, DriveRecess, Material, Grade, Length, Diameter, ThreadType, ThreadPitch, TensileStrength_PSI_, Color, Application, BandWidth, ScrewSize, HeadHeight, ThreadLength, FlangeDiameter, WidthAcrosstheFlats, UsableLength, OverallLength, GripRange, BodyMaterial, HeadDiameter, MandrelMaterial, HoleSize, Size, HeadSize, StemLength, StemSize, IndustryNumber, OutsideDiameter, ThreadSize, HoleDiameter, Width, Style, Polarity, StudSize, InsideDiameter, Height, TubeSize, BoltSize, PipeSize, ClampRange, HoseO_D_, HoseI_D_, Shape, MaxPressure, SealingMethod, MalePipeThread, VibrationRating, FemalePipeThread, OperatingTemperature, SocketSize, RatedCapacity, ChainSize, Bore, BundleRange, mm_name, mm_ld, mm_hd, head, hd_ld, Q:head_diameter_sae]
Index: []

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcurate))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))


# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchex))

matchcurate
matchna
matchex


In [12]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [15]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcurate) 

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [17]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchex) 